In [1]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

In [9]:
import streamlit as st

In [4]:
class app:
  def __init__(self,host,user,port,password,database):
    self.host=host
    self.user=user
    self.port=port
    self.password=password
    self.database=database
    self.conn=None
    self.cursor=None
  def connect(self):
    self.conn=mysql.connector.connect(host=self.host,user=self.user,port=self.port,password=self.password,database=self.database)
    self.cursor=self.conn.cursor()
    print("coonection was suceed")
  def create_table(self,t_query):
    self.cursor.execute(t_query)
    self.conn.commit()
    print("table created")
  def insert_data(self,I_query,list_of_tuples):
    self.cursor.executemany(I_query,list_of_tuples)
    self.conn.commit()
    print("value inserted")
  def reconnect(self):
        try:
            self.conn.ping(reconnect=True)  # Attempt to reconnect if connection is lost
            print("Reconnected to the database.")
        except mysql.connector.Error as err:
            print(f"Error reconnecting: {err}")

In [5]:
config = {"host" : "gateway01.ap-southeast-1.prod.aws.tidbcloud.com", "user": "Ud6xHKHHCuFbH6f.root", "port" : 4000, "password" : "jwl7QF0l1usUQwNi","database":"Zomato"}

In [6]:
df_obj= app(**config)

In [7]:
df_obj.connect()


coonection was suceed


In [ ]:
df_obj.reconnect()

Reconnected to the database.


In [8]:
%pip install faker --upgrade  

from faker import Faker
from datetime import datetime, timedelta
fake = Faker(["en_IN"])

Note: you may need to restart the kernel to use updated packages.


In [ ]:
EMP_ID=[]
cuisines = [
    "Indian", "Chinese", "Italian", "Mexican", "Thai", "Japanese", "Mediterranean",
    "American", "French", "Korean", "Vietnamese", "Greek", "Spanish", "Lebanese"]
for i in range(1,1000):
  EMP_data=(fake.unique.random_int(min=1000 ,max=9999),fake.name(),fake.unique.email(),
            fake.phone_number(),fake.address(), fake.date_between(start_date= datetime(2020,1,1), end_date=datetime(2024,12,31)),
            fake.random_element(["Yes","No"]),fake.random_element(cuisines),fake.random_int(min=1, max=100),round(fake.random.uniform(0,5),2))
  EMP_ID.append(EMP_data)

In [8]:
tq="""create table Customer_Table(Customer_ID int primary key,Customer_Name varchar(200),
      Customer_Email varchar(200),Customer_Phno varchar(15),Customer_Address varchar(400),
      Customer_Signup_date DATE,Is_Premium varchar(20),Preferred_Cuisine varchar(20),Total_orders int,
      Average_rating float)"""
df_obj.create_table(tq)

table created


In [ ]:
Iq="""insert into Customer_Table(Customer_ID,Customer_Name,Customer_Email,Customer_Phno,Customer_Address,Customer_Signup_date,Is_Premium,Preferred_Cuisine,Total_orders, Average_rating )
    values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
df_obj.insert_data(Iq,EMP_ID)

In [ ]:
RT=[]
for _ in range(1,1000):
  delivery_times = [fake.date_time_this_month().replace(second=0) for _ in range(5)]
  reference_time = datetime.combine(delivery_times[0].date(), datetime.min.time())
  minutes_list = [(dt - reference_time).total_seconds() / 60 for dt in delivery_times]
  average_delivery_time = sum(minutes_list) / len(minutes_list)
  average_minute = int(average_delivery_time % 60)

  rt_tab =  (
         fake.bothify(text="?#####"),
         fake.company() + " " + fake.random_element(["Grill", "Cafe", "Bistro", "Diner", "Eatery"]),
         fake.random_element(cuisines),fake.address(),fake.name(),abs(average_delivery_time),
         fake.phone_number(),
         round(fake.random.uniform(1,5),2),
         fake.random_int(min=1,max=100),
         fake.random_element(["Yes","No"])
            )
  RT.append(rt_tab)

In [13]:
rq="""create table Restaurant_Table(Restaurant_ID varchar(200) primary key,Restaurant_Name varchar(200),
      Cusine_type varchar(200),Reataurant_location varchar(200),Owner_Name varchar(40),
      Avg_delivery_time float,Contact_number varchar(20), Average_rating float ,Total_orders int,
      Is_Active varchar(10))"""
df_obj.create_table(rq)

table created


In [ ]:
rest="""insert into Restaurant_Table(Restaurant_ID,Restaurant_Name,Cusine_type,Reataurant_location,Owner_Name,Avg_delivery_time,Contact_number,Average_rating,Total_orders,Is_Active)
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
df_obj.insert_data(rest,RT)


In [10]:
df_obj.cursor.execute("select Customer_ID from Customer_Table")
Customer_ids=[row[0] for row in df_obj.cursor.fetchall()]

In [ ]:
df_obj.cursor.execute("select Restaurant_ID from Restaurant_Table")
Restaurant_ids=[row[0] for row in df_obj.cursor.fetchall()]

In [ ]:
OT = []
for _ in range(1,1000):  # Fixed range
    delivery_offset = fake.random.randint(10, 120)
    order_amount = fake.random.randint(500, 5000)  # Order amount between ₹500 - ₹5000
    discount_percentage = fake.random.randint(5, 50)  # Discount between 5% - 50%
    discount_amount = (discount_percentage / 100) * order_amount  # Calculate discount

    # Generate random datetime
    random_datetime = fake.date_time_between(start_date="-4y", end_date="now")  # Proper datetime usage

    # Add delivery offset using timedelta
    delivery_datetime = random_datetime + timedelta(minutes=delivery_offset)

    order = (
        fake.unique.random_int(min=10000, max=99999),  # Removed .unique to avoid errors
        fake.random.choice(Customer_ids),
        fake.random.choice(Restaurant_ids),
        random_datetime,
        delivery_datetime,
        fake.random_element(["Pending", "Delivered", "Cancelled"]),
        round(order_amount, 2),
        fake.random_element(["Credit Card", "Cash", "UPI"]),
        round(discount_amount, 2),
        round(fake.random.uniform(1, 5), 1),
    )
    OT.append(order)


In [17]:
order="""create table Order_Table(Order_ID int primary key,Customer_ID int,Restaurant_ID varchar(200),Order_Date DATETIME,Delivery_Time DATETIME,
         Order_Status varchar(20),Order_Amount float,Payment_Mode varchar(20),Discount_Amount float,Rating float,
         FOREIGN KEY (Customer_ID) REFERENCES Customer_Table(Customer_ID) ON DELETE CASCADE,
         FOREIGN KEY (Restaurant_ID) REFERENCES Restaurant_Table(Restaurant_ID) ON DELETE CASCADE)"""
df_obj.create_table(order)

table created


In [24]:
order_val="""insert into Order_Table(Order_ID,Customer_ID,Restaurant_ID,Order_Date,Delivery_Time,Order_Status,Order_Amount,Payment_Mode,Discount_Amount,Rating)
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

df_obj.insert_data(order_val, OT) 

value inserted


In [ ]:
DPT=[]
vehicle_types = ["Bike", "Car", "Scooter", "Van", "Truck", "Bicycle"]
for _ in range(1,1000):
  person =  (
         fake.unique.random_int(min=1000 ,max=9999),
         fake.name(),
         fake.phone_number(),
         fake.random_element(vehicle_types),
         fake.random_int(min=1, max=100),
         round(fake.random.uniform(1, 5), 1),
         fake.address()
      )
  DPT.append(person)

In [25]:
person_tble="""create table Delivery_Person_Table(Delivery_Person_ID int primary key,Person_Name varchar(200),
               Person_Phno varchar(20),Vechicle_type varchar(20),Total_Deliveries int,
               Average_rating float,Person_Address varchar(200))"""
df_obj.create_table(person_tble)

table created


In [27]:
pt="""insert into Delivery_Person_Table(Delivery_Person_ID,Person_Name,Person_Phno,Vechicle_type,Total_Deliveries,Average_rating,Person_Address)
      values(%s,%s,%s,%s,%s,%s,%s)"""
df_obj.insert_data(pt, DPT)  # Insert into Delivery_Person_Table first

value inserted


In [ ]:
df_obj.cursor.execute("""
    SELECT Order_ID
    FROM Order_Table
    WHERE Order_ID IN (SELECT Order_ID FROM Order_Table)
""")
order_ids = [row[0] for row in df_obj.cursor.fetchall()]

In [ ]:
df_obj.cursor.execute("""
    SELECT Delivery_Person_ID
    FROM Delivery_Person_Table
    WHERE Delivery_Person_ID IN (SELECT Delivery_Person_ID FROM Delivery_Person_Table)
""")
Delivery_Person_ids = [row[0] for row in df_obj.cursor.fetchall()]


In [ ]:
import random
DT=[]
for _ in range(1,1000):
  order_time = fake.date_time_between(start_date='-30d', end_date='now')
  estimated_time = order_time + timedelta(minutes=random.randint(30, 60))

    # Generate Actual Delivery Time (could be late, e.g., 10-20 minutes later)
  actual_delivery_time = estimated_time + timedelta(minutes=random.randint(0, 20))

  deli = (
      fake.unique.random_int(min=10000, max=99999),
      fake.random.choice(order_ids),
      fake.random.choice(Delivery_Person_ids),
      fake.random_element(["On the way","Delivered","In Transit","Pending","Out for Delivery","Cancelled"]),
      fake.random_int(min=1, max=100),
      actual_delivery_time,
      estimated_time ,
      fake.random_int(min=10, max=100),
      fake.random_element(vehicle_types)

  )
  DT.append(deli)

In [28]:
delivery_table="""create table Delivery_Table(Delivery_ID int primary key,Order_ID int ,Delivery_Person_ID int,Delivery_Status varchar(20),
                  Distance_to_Deliver int,Delivery_Time DATETIME,Estimated_Delivery_Time DATETIME,Delivery_Fee float,Vehicle_type varchar(20),
                  FOREIGN KEY (Order_ID) REFERENCES Order_Table(Order_ID) ON DELETE CASCADE,
                  FOREIGN KEY (Delivery_Person_ID) REFERENCES Delivery_Person_Table(Delivery_Person_ID) ON DELETE CASCADE)"""

df_obj.create_table(delivery_table)

table created


In [32]:
idt="""insert into Delivery_Table(Delivery_ID,Order_ID,Delivery_Person_ID,Delivery_Status,Distance_to_Deliver,Delivery_Time,
        Estimated_Delivery_Time,Delivery_Fee,Vehicle_type)
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
df_obj.insert_data(idt, DT)

value inserted


In [ ]:
df_obj.reconnect()

Reconnected to the database.


In [ ]:
from datetime import datetime
random_date = fake.date_between(start_date= datetime(2020,1,1), end_date=datetime(2024,12,31))
print(random_date)

2022-01-02
